# Image Logging

Whylogs has optional support for logging data about images. This example demonstrates how to use whylogs with image data.

## Install whylogs with Image Logging Support

Whylogs uses [Pillow](https://python-pillow.org) for working with images. To install whylogs with the optional image support enabled, use the following:








In [ ]:
pip install whylogs[image]

This will install all the dependencies whylogs needs to work with image data.

## Basic Image Logging

The `log_image()` function provides a simple interface for logging images.

In [ ]:
import os
from PIL import Image

from whylogs.extras.image_metric import log_image

TEST_DATA_PATH = os.path.abspath(
    os.path.join(
        os.path.realpath(os.path.dirname(__file__)),
        os.pardir, os.pardir,
        "testdata", "images"
    )
)

def load_image(file_name: str) -> Image.Image:
  with open(os.path.join(TEST_DATA_PATH, file_name), "rb") as file_p:
    return Image.open(file_p).copy()

results = log_image(load_image("flowers2.jpg"))
print(results.view().get_column("image").to_summary_dict())

As you can see above, just passing in an `Image` results in a "column" named in the profile. You can pass in a list of images, which will append an index to each column name:

In [ ]:
img = load_image("flowers2.jpg")
results = log_image([img, img, img])
print(results.view().get_column("image2").to_summary_dict())

You can change the default name if you like:

In [ ]:
results = log_image([img, img], default_column_prefix="awesome_image")
print(results.view().get_column("awesome_image0").to_summary_dict())

You can also pass a dictionary of images to give each "column" a unique name:

In [ ]:
results = log_image({"red": img, "blue": img, "green": image})
print(results.view().get_column("blue").to_summary_dict())

## Advanced Image Logging

The `log_image()` function provides a flexible interface for logging image data. If you want to log images along with other types of data, you can do so by setting up a custom `DatasetSchema` to specify which columns contain images.

In [ ]:
import whylogs as why

from whylogs.core.metrics import MetricConfig
from whylogs.core.resolvers import StandardResolver
from whylogs.core.schema import DatasetSchema
from whylogs.extras.image_metric import ImageMetric

class ImageResolver(StandardResolver):
  def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
    if "image" in name:
      return {ImageMetric.get_namespace(MetricConfig()): ImageMetric.zero(column_schema.cfg)}
    return super(ImageResolver, self).resolve(name, column_schema)

schema = DatasetSchema(resolvers=ImageResolvler())

results = why.log(row={"numbers": 42, "images": img}, schema=schema)
print(results.to_summary_dict())